[loto7 winning number page](https://www.mizuhobank.co.jp/retail/takarakuji/check/loto/loto7/index.html)

total_number = 全通りの数

In [2]:
# ライブラリのインポート
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt # データを効率的に操作するためのライブラリ
import numpy as np              # グラフを簡単に書くライブラリ
import os
import difflib

In [58]:
# ファイル配置場所を変数に入れて読み取り専用で開く
file_path = "./Billion.xlsx"
wb = openpyxl.load_workbook(file_path, data_only=True)

In [59]:
# ロト7の総当選数(37C7 = 10295472.0)
total_number = (37 * 36 * 35 * 34 * 33 * 32 * 31) / (7 * 6 * 5 * 4 * 3 * 2 * 1)
#[debug] print(total_number)

In [60]:
# 抽選回数の最終値を抽出する
ws = wb["当選番号"]
last_row = ws.max_row
#[debug] print(last_row)

In [61]:
# 重複チェック
def check_duplicates(new_data, existing_data):
    for key, values in existing_data.items():
        if new_data == values:
            return True  # 重複が見つかった場合
    return False

# 抽選回数分の空データフレームを作成する
# ref: 6.1 whileによる反復処理
count = 1
win_numbers = {}  # 辞書を初期化

while count <= last_row:
    win_num_name = "win_num" + str(count)
    #[debug] print(win_num_name)

    # cell番号を指定して値を要素に追加する
    b_value = ws["B" + str(count)].value
    c_value = ws["C" + str(count)].value
    d_value = ws["D" + str(count)].value
    e_value = ws["E" + str(count)].value
    f_value = ws["F" + str(count)].value
    g_value = ws["G" + str(count)].value
    h_value = ws["H" + str(count)].value
    
     # Noneの場合に備えて条件を追加
    if None not in (b_value, c_value, d_value, e_value, f_value, g_value, h_value):
        win_numbers[win_num_name] = {
        'no1': int(b_value),
        'no2': int(c_value),
        'no3': int(d_value),
        'no4': int(e_value),
        'no5': int(f_value),
        'no6': int(g_value),
        'no7': int(h_value)
        }
    count += 1

# 特定のwin_numのデータを出力
#[debug] print(win_numbers['win_num2'])

In [68]:
# 重複をチェックするためのセット
seen_numbers = set()

# 重複があるかどうかのフラグ
has_duplicates = False

# 重複チェック処理
for key, value in win_numbers.items():
    numbers_tuple = tuple(value.values()) # key=win_num1 value={'no1': 7, 'no2': 10, 'no3': 12, 'no4': 17, 'no5': 23, 'no6': 28, 'no7': 34}
    #[debug] print(numbers_tuple) # 出力 -> (7, 10, 12, 17, 23, 28, 34)    

    if numbers_tuple in seen_numbers:
        # 重複がある場合
        has_duplicates = True
        print(f"重複したデータが見つかりました: {key} - {value}")
    else:
        # 重複がない場合、セットに追加
        seen_numbers.add(numbers_tuple)

#if not has_duplicates:
#    print("重複したデータはありません。") # 重複したときだけメッセージ出したいためコメントアウト

# wn_list.txtの初期化
with open("wn_list.txt", mode="r+") as f:  # rwでファイルを開くこと
    f.truncate(0)

# wn_list.txt(当選番号のみのリスト)を作成
with open('wn_list.txt', 'w') as f:
    for wn_list in seen_numbers:
        f.write(', '.join(map(str, wn_list)) + '\n') # リストの各要素を文字列に変換し、カンマで区切って一つの文字列に結合している。

#[debug] print(seen_numbers)

TypeError: 'tuple' object is not callable

In [67]:
# 重複してなかったらtotal_numberから回数分引く。
total_number_latest = total_number - last_row
#[debug] 
print(total_number_latest)

10294898.0


In [9]:
# 全no1の合計値
count = 1
sum_no1 = 0

while count <= last_row:
    no1_val = (win_numbers['win_num' + str(count)]['no1'])
    sum_no1 = sum_no1 + no1_val
    count += 1

# output(result)
#[debug] print(sum_no1)

# no1の平均値の出力
avg_no1 = None
globals()['avg_no1'] = sum_no1 / last_row
#[debug] print(avg_no1)

In [10]:
# no2の平均値
count = 1
sum_no2 = 0

while count <= last_row:
    no2_val = (win_numbers['win_num' + str(count)]['no2'])
    sum_no2 = sum_no2 + no2_val
    count += 1

# output(result)
#[debug] print(sum_no2)

# no2の平均値の出力
avg_no2 = None
globals()['avg_no2'] = sum_no2 / last_row

In [11]:
# no3の平均値
count = 1
sum_no3 = 0

while count <= last_row:
    no3_val = (win_numbers['win_num' + str(count)]['no3'])
    sum_no3 = sum_no3 + no3_val
    count += 1

# output(result)
#[debug] print(sum_no3)

# no3の平均値の出力
avg_no3 = None
globals()['avg_no3'] = sum_no3 / last_row

In [12]:
# no4の平均値
count = 1
sum_no4 = 0

while count <= last_row:
    no4_val = (win_numbers['win_num' + str(count)]['no4'])
    sum_no4 = sum_no4 + no4_val
    count += 1

# output(result)
#[debug] print(sum_no4)

# no4の平均値の出力
avg_no4 = None
globals()['avg_no4'] = sum_no4 / last_row

In [13]:
# no5の平均値
count = 1
sum_no5 = 0

while count <= last_row:
    no5_val = (win_numbers['win_num' + str(count)]['no5'])
    sum_no5 = sum_no5 + no5_val
    count += 1

# output(result)
#[debug] print(sum_no5)

# no5の平均値の出力
avg_no5 = None
globals()['avg_no5'] = sum_no5 / last_row

In [14]:
# no6の平均値
count = 1
sum_no6 = 0

while count <= last_row:
    no6_val = (win_numbers['win_num' + str(count)]['no6'])
    sum_no6 = sum_no6 + no6_val
    count += 1

# output(result)
#[debug] print(sum_no6)

# no6の平均値の出力
avg_no6 = None
globals()['avg_no6'] = sum_no6 / last_row

In [15]:
# no7の平均値
count = 1
sum_no7 = 0

while count <= last_row:
    no7_val = (win_numbers['win_num' + str(count)]['no7'])
    sum_no7 = sum_no7 + no7_val
    count += 1

# output(result)
#[debug] print(sum_no7)

# no7の平均値の出力
avg_no7 = None
globals()['avg_no7'] = sum_no7 / last_row
#[debug] print(globals()['avg_no7'])

In [16]:
# 全Noのavgを出力する。
count = 1
while count <= 7:
    no_name = 'avg_no' + str(count)
    print(f'{globals()[no_name]}')
    count += 1

5.087813620071684
9.666666666666666
14.304659498207885
18.95878136200717
23.845878136200717
28.668458781362006
33.270609318996414


In [ ]:
# [important]:第1-10回までの法則性を見つけ第11回の値が算出できるか。
count = 1
sum_no1 = 0

while count <= 10:
    no1_val = (win_numbers['win_num' + str(count)]['no1'])
    sum_no1 = sum_no1 + no1_val
    count += 1

# 1-10の合計値
print(sum_no1)

# 1-10の平均値（result=5.4）
print(sum_no1 / 10)

# 第1オクテットだけを出力させたい場合
## 最頻値（Mode）: 全n回のデータセット内で最も頻繁に出現する値
#first_values = [taple[0] for taple in seen_numbers]

# 第1~6オクテットまでをbuik出力
count = 0
while count <= 6:
    first_values = [taple[count] for taple in seen_numbers]
    
    ## 辞書内包表記を使って1個目の数値が何回現れたのかを数える。
    first_values_count = {first: first_values.count(first) for first in first_values }

    ## 最頻値の数(valueが大きいkey)を出力させる。
    max_key = max(first_values_count, key=first_values_count.get)
    max_value = first_values_count[max_key]
    print(f"第{count + 1}オクテットの最頻値は{max_key}で、{max_value}回")

    ## [debug]昇順で全出力させる。
    #for key in sorted(first_values_count.keys()): # sorted()関数でキーを昇順にソート
    #    print(f"{key}: {first_values_count[key]}") # ソートされたキーを使ってループを回し、各キーと対応する値を昇順で出力
 
    count += 1

# 中央値（Median）: データを昇順に並べたときの中央値。データが偶数個の場合は中央の2つの値の平均を取る

# win_num11のwinningnumberは11であることを算出する。
plt.title("loto7 is mine.")           # タイトル

# y
plt.ylabel("winning number")          # 縦のタイトル
y = np.array([7,20,2,12,1,5,1,2,3,1]) # win_num1-10までのwinningnumvber

# x
plt.xlabel("times")                   # 横のタイトル
x = np.array([1,2,3,4,5,6,7,8,9,10])  # 回数

# graph出力
plt.plot(x, y)
plt.show()

In [54]:
# 1から37までの数字を7つ列挙する全ての組み合わせを生成するジェネレータ
from itertools import combinations

# combinations.txtの初期化
with open("combinations.txt", mode="r+") as f:  # rwでファイルを開くこと
    f.truncate(0)

def generate_combinations():
    numbers = list(range(1, 38))
    for combo in combinations(numbers, 7):
        yield combo

# ファイルへの書き込み
with open('combinations.txt', 'w') as file:
    # ジェネレータを使用して組み合わせを順次取得し、ファイルに書き込む
    combinations_generator = generate_combinations()
    for combination in combinations_generator:
        # 組み合わせを文字列に変換してファイルに書き込み
        line = ', '.join(map(str, combination)) + '\n'
        file.write(line)

In [55]:
# combinations.txtからwn_list.txtを削除
# reference : https://python-academia.com/file-comparison/
try:
    with open('./combinations.txt') as open_combination, open('./wn_list.txt') as open_wn_list:
        # Differオブジェクトを生成
        diff = difflib.Differ()

        # 差分生成
        output_diff = diff.compare(open_combination.readlines(), open_wn_list.readlines())

    # result.txtの初期化
    with open("result.txt", mode="r+") as f:  # rwでファイルを開くこと
        f.truncate(0)

    with open('result.txt', 'w') as file:
        for data in output_diff:
            # 行がwn_list.txtに存在しない場合にのみ書き込み
            if data.startswith('-') and not any(char.isdigit() for char in data):
                file.write(data[2:])  # '-'を取り除いて書き込み
            
except FileNotFoundError as e:
    print(f"エラー: {e.filename} が見つかりませんでした。")
except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

In [ ]:
# combinations.txtからwn_list.txtを削除
with open('./combinations.txt') as open_combination, open('./wn_list.txt') as open_wn_list:
    # combinations.txt の各行をセットに変換
    combinations_set = set(open_combination.read().splitlines())

    # wn_list.txt の各行をセットに変換
    wn_list_set = set(open_wn_list.read().splitlines())

# result.txtの初期化
with open("result.txt", mode="w") as f:
    f.truncate(0)

# wn_list.txtに存在しない組み合わせをresult.txtに書き込む
with open('result.txt', 'w') as file:
    for combination in combinations_set - wn_list_set:
        file.write(combination + '\n')

In [ ]:
# no1が奇数の時、no2は奇数？偶数？

In [45]:
# no1で出ていない数
with open('wn_list.txt', 'r') as f:
    numbers = f.readlines().rstrip("\n")
print(numbers)

AttributeError: 'list' object has no attribute 'rstrip'